In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
#from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import gc

In [2]:
origin_data = pd.read_csv('winemag-data-130k-v2.csv\winemag-data-130k-v2.csv')

In [3]:
data=origin_data.dropna(axis=0,subset=['price'])

In [4]:
data=data.drop(data.columns[0],axis=1)

In [5]:
data.reset_index(drop=True, inplace=True)

In [65]:
data

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
5,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
6,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
7,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
8,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
9,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature


In [7]:
ncol=[]
ncol.append(data.columns)
ncol.append([data[col].nunique() for col in data.columns])
ncol=pd.DataFrame(np.asarray(ncol))
ncol

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,42,111567,35776,21,390,422,1204,17,19,15,110638,697,15855


In [8]:
missing_df = data.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df

,column_name,missing_count
0,country,59
1,description,0
2,designation,34779
3,points,0
4,price,0
5,province,59
6,region_1,19575
7,region_2,70683
8,taster_name,24496
9,taster_twitter_handle,29416


In [9]:
data['title']

0             Quinta dos Avidagos 2011 Avidagos Red (Douro)
1             Rainstorm 2013 Pinot Gris (Willamette Valley)
2         St. Julian 2013 Reserve Late Harvest Riesling ...
3         Sweet Cheeks 2012 Vintner's Reserve Wild Child...
4         Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...
5          Terre di Giurfo 2013 Belsito Frappato (Vittoria)
6                     Trimbach 2012 Gewurztraminer (Alsace)
7         Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...
8         Jean-Baptiste Adam 2012 Les Natures Pinot Gris...
9         Kirkland Signature 2011 Mountain Cuvée Caberne...
10                  Leon Beyer 2012 Gewurztraminer (Alsace)
11        Louis M. Martini 2012 Cabernet Sauvignon (Alex...
12                 Mirassou 2012 Chardonnay (Central Coast)
13              Richard Böcking 2013 Devon Riesling (Mosel)
14               Felix Lavaque 2010 Felix Malbec (Cafayate)
15        Gaucho Andino 2011 Winemaker Selection Malbec ...
16        Pradorey 2010 Vendimia Selecci

In [7]:

def extract_title(title):
    n = len(title)
    
    ex_title1 = None
    ex_title2 = None
    ex_title3 = None
    ex_title4 = None
    if title == None :
        return ex_title1,ex_title2,ex_title3,ex_title4
    ye=99999
    i = n-3
    while i in range(n):
        if (title[i]=='2')or(title[i]=='1')and(i+4<=n) :
            st = title[i:i+4]
            if st.isdigit():
                if (int(st)>1948)and(int(st)<2018):
                    ye = i
                    i = 99999
        i -= 1
    if ye != 99999:
        ex_title1 = title[:ye]
        ex_title2 = title[ye:ye+4]
        title = title[ye+4:]
    n = len(title)
    i = n-1
    brac=99999
    if title[i]==')':  
        while i in range(n):
            if title[i]=='(':
                brac = i
                i = 99999
            i -= 1
    else:
        ex_title3=title[0:]
    if brac != 99999:
        ex_title3 = title[0:brac]
        ex_title4=title[brac+1:n-1]
    
    ex_title1 = ex_title1.strip() if ex_title1 != None else None
    ex_title2 = ex_title2.strip() if ex_title2 != None else None
    ex_title3 = ex_title3.strip() if ex_title3 != None else None
    ex_title4 = ex_title4.strip() if ex_title4 != None else None
    return ex_title1,ex_title2,ex_title3,ex_title4

In [66]:
title_df = pd.DataFrame(np.asarray([extract_title(title) for title in data['title']]),columns=['ex_title1','year','ex_title2','ex_title3'])

In [67]:
data=data.join(title_df)

In [15]:
data

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,ex_title1,year,ex_title2,ex_title3
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos,2011,Avidagos Red,Douro
1,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Rainstorm,2013,Pinot Gris,Willamette Valley
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,St. Julian,2013,Reserve Late Harvest Riesling,Lake Michigan Shore
3,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Sweet Cheeks,2012,Vintner's Reserve Wild Child Block Pinot Noir,Willamette Valley
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,Tandem,2011,Ars In Vitro Tempranillo-Merlot,Navarra
5,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,Terre di Giurfo,2013,Belsito Frappato,Vittoria
6,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,Trimbach,2012,Gewurztraminer,Alsace
7,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,Heinz Eifel,2013,Shine Gewürztraminer,Rheinhessen
8,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,Jean-Baptiste Adam,2012,Les Natures Pinot Gris,Alsace
9,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Kirkland Signature,2011,Mountain Cuvée Cabernet Sauvignon,Napa Valley


In [16]:
missing_df = data.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
#missing_df = missing_df.sort_values(by='missing_count')
missing_df

,column_name,missing_count
0,country,59
1,description,0
2,designation,34779
3,points,0
4,price,0
5,province,59
6,region_1,19575
7,region_2,70683
8,taster_name,24496
9,taster_twitter_handle,29416


In [68]:
data['year'] =  data['year'].apply(pd.to_numeric)

### Region

In [18]:
data[data['ex_title3'].isnull()]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,ex_title1,year,ex_title2,ex_title3
58,France,"From the warm 2015 vintage, this is a soft and...",NaN,86,24.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,Simonnet-Febvre 2015 Chablis,Chardonnay,Simonnet-Febvre,Simonnet-Febvre,2015.0,Chablis,None
59,France,"This soft, rounded wine is ripe with generous ...",NaN,86,15.0,Burgundy,Mâcon-Milly Lamartine,NaN,Roger Voss,@vossroger,Vignerons des Terres Secrètes 2015 Mâcon-Mill...,Chardonnay,Vignerons des Terres Secrètes,Vignerons des Terres Secrètes,2015.0,Mâcon-Milly Lamartine,None
87,France,"This is a dense wine, packed with both tannins...",NaN,88,20.0,Beaujolais,Juliénas,NaN,Roger Voss,@vossroger,Henry Fessy 2015 Juliénas,Gamay,Henry Fessy,Henry Fessy,2015.0,Juliénas,None
88,France,The wine comes from one of the cru estates fol...,NaN,88,18.0,Beaujolais,Régnié,NaN,Roger Voss,@vossroger,Henry Fessy 2015 Régnié,Gamay,Henry Fessy,Henry Fessy,2015.0,Régnié,None
109,US,"Baked red cherry, crushed clove, iron and rose...",Estate,87,44.0,California,Central Coast,NaN,Matt Kettmann,@mattkettmann,Rideau 2014 Estate Syrah,Syrah,Rideau,Rideau,2014.0,Estate Syrah,None
125,Italy,"Einaudi's wines have been improving lately, an...",NaN,91,68.0,Piedmont,Barolo,NaN,NaN,NaN,Poderi Luigi Einaudi 2003 Barolo,Nebbiolo,Poderi Luigi Einaudi,Poderi Luigi Einaudi,2003.0,Barolo,None
128,France,This wine's black cherry and plum flavors veer...,NaN,91,50.0,Bordeaux,Saint-Émilion,NaN,Joe Czerwinski,@JoeCz,Lassègue 2003 Saint-Émilion,Bordeaux-style Red Blend,Lassègue,Lassègue,2003.0,Saint-Émilion,None
166,Italy,This Chianti Classico strikes a beautiful bala...,NaN,88,22.0,Tuscany,Chianti Classico,NaN,NaN,NaN,Isole e Olena 2005 Chianti Classico,Sangiovese,Isole e Olena,Isole e Olena,2005.0,Chianti Classico,None
181,France,"A citrus-dominated wine, lime and lemon giving...",NaN,87,35.0,Burgundy,St.-Romain,NaN,Roger Voss,@vossroger,Bertrand Ambroise 2006 St.-Romain,Chardonnay,Bertrand Ambroise,Bertrand Ambroise,2006.0,St.-Romain,None
202,Italy,"Black-skinned berry, pipe tobacco and Mediterr...",NaN,90,57.0,Tuscany,Brunello di Montalcino,NaN,Kerin O’Keefe,@kerinokeefe,Podere Scopetone 2012 Brunello di Montalcino,Sangiovese,Podere Scopetone,Podere Scopetone,2012.0,Brunello di Montalcino,None


In [69]:
ii = data[data['ex_title3'].isnull()].index

tmp=0
for i in ii:
    tmp = data.loc[i,'region_1']
    if tmp is not np.nan:
        data.loc[i,'ex_title3']=tmp
    else:
        tmp = data.loc[i,'province']
        if tmp is not np.nan:
            data.loc[i,'ex_title3']=tmp         

In [ ]:
data=data.drop(['taster_twitter_handle','province','region_1','region_2','ex_title1','ex_title2'],axis=1)

In [21]:
data

,country,description,designation,points,price,taster_name,title,variety,winery,year,ex_title3
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0,Douro
1,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013.0,Willamette Valley
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013.0,Lake Michigan Shore
3,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012.0,Willamette Valley
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011.0,Navarra
5,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013.0,Vittoria
6,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012.0,Alsace
7,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,2013.0,Rheinhessen
8,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012.0,Alsace
9,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,Virginie Boone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,2011.0,Napa Valley


### Description

In [7]:
num_words=200

In [8]:
X=data['description'].tolist()
X=[x.lower() for x in X]
Y=data['price'].tolist()

In [9]:
X_tokens = [word_tokenize(x) for x in X]
stop_words = set(stopwords.words('english'))

In [10]:
ps=PorterStemmer()
def st(x):
    new_x=[ps.stem(stw) for stw in  x]
    return [xx for xx in new_x if xx not in stop_words]  

In [11]:
X_del_st = [st(x) for x in X_tokens]

In [40]:
import gc
gc.collect()

28

In [19]:
all_words = []

for row in X_del_st:
    for w in row:
        all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

In [20]:
word_features = dict(sorted(all_words.items(),key=lambda item:item[1],reverse=True)[:num_words]).keys()
word_features_dict = dict(zip(word_features,range(len(word_features))))

In [21]:
# create featuresets
def find_features(review):
    features = [0]*num_words
    for w in review:
        if w in word_features_dict.keys():
            features[word_features_dict[w]] += 1
    return features

final_X = [find_features(rev) for rev in X_del_st]

In [22]:
bow=pd.DataFrame(final_X,columns=[f+'_nlp' for f in word_features])

In [71]:
data=data.join(bow)

In [72]:
data=data.drop(['description','title',',_nlp','._nlp','thi_nlp','\'s_nlp'],axis=1)

In [73]:
data

,country,designation,points,price,taster_name,variety,winery,year,ex_title3,wine_nlp,...,seem_nlp,cranberri_nlp,lightli_nlp,plenti_nlp,toasti_nlp,perfum_nlp,quit_nlp,doe_nlp,petit_nlp,tangi_nlp
0,Portugal,Avidagos,87,15.0,Roger Voss,Portuguese Red,Quinta dos Avidagos,2011.0,Douro,1,...,0,0,0,0,0,0,0,0,0,0
1,US,NaN,87,14.0,Paul Gregutt,Pinot Gris,Rainstorm,2013.0,Willamette Valley,1,...,0,0,0,0,0,0,0,0,0,0
2,US,Reserve Late Harvest,87,13.0,Alexander Peartree,Riesling,St. Julian,2013.0,Lake Michigan Shore,0,...,0,0,0,0,0,0,0,0,0,0
3,US,Vintner's Reserve Wild Child Block,87,65.0,Paul Gregutt,Pinot Noir,Sweet Cheeks,2012.0,Willamette Valley,1,...,0,0,0,0,0,0,0,0,0,0
4,Spain,Ars In Vitro,87,15.0,Michael Schachner,Tempranillo-Merlot,Tandem,2011.0,Navarra,0,...,0,0,0,0,0,0,0,0,0,0
5,Italy,Belsito,87,16.0,Kerin O’Keefe,Frappato,Terre di Giurfo,2013.0,Vittoria,0,...,0,0,0,0,0,0,0,0,0,0
6,France,NaN,87,24.0,Roger Voss,Gewürztraminer,Trimbach,2012.0,Alsace,1,...,0,0,0,0,0,0,0,0,0,0
7,Germany,Shine,87,12.0,Anna Lee C. Iijima,Gewürztraminer,Heinz Eifel,2013.0,Rheinhessen,1,...,0,0,0,0,0,0,0,0,0,0
8,France,Les Natures,87,27.0,Roger Voss,Pinot Gris,Jean-Baptiste Adam,2012.0,Alsace,0,...,0,0,0,0,0,0,0,0,0,0
9,US,Mountain Cuvée,87,19.0,Virginie Boone,Cabernet Sauvignon,Kirkland Signature,2011.0,Napa Valley,1,...,0,0,0,0,0,0,0,0,0,0


In [74]:
data_copy=data.copy()

In [35]:
data_copy

,country,designation,points,price,taster_name,variety,winery,year,ex_title3,wine_nlp,...,seem_nlp,cranberri_nlp,lightli_nlp,plenti_nlp,toasti_nlp,perfum_nlp,quit_nlp,doe_nlp,petit_nlp,tangi_nlp
0,Portugal,Avidagos,87,15.0,Roger Voss,Portuguese Red,Quinta dos Avidagos,2011.0,Douro,1,...,0,0,0,0,0,0,0,0,0,0
1,US,NaN,87,14.0,Paul Gregutt,Pinot Gris,Rainstorm,2013.0,Willamette Valley,1,...,0,0,0,0,0,0,0,0,0,0
2,US,Reserve Late Harvest,87,13.0,Alexander Peartree,Riesling,St. Julian,2013.0,Lake Michigan Shore,0,...,0,0,0,0,0,0,0,0,0,0
3,US,Vintner's Reserve Wild Child Block,87,65.0,Paul Gregutt,Pinot Noir,Sweet Cheeks,2012.0,Willamette Valley,1,...,0,0,0,0,0,0,0,0,0,0
4,Spain,Ars In Vitro,87,15.0,Michael Schachner,Tempranillo-Merlot,Tandem,2011.0,Navarra,0,...,0,0,0,0,0,0,0,0,0,0
5,Italy,Belsito,87,16.0,Kerin O’Keefe,Frappato,Terre di Giurfo,2013.0,Vittoria,0,...,0,0,0,0,0,0,0,0,0,0
6,France,NaN,87,24.0,Roger Voss,Gewürztraminer,Trimbach,2012.0,Alsace,1,...,0,0,0,0,0,0,0,0,0,0
7,Germany,Shine,87,12.0,Anna Lee C. Iijima,Gewürztraminer,Heinz Eifel,2013.0,Rheinhessen,1,...,0,0,0,0,0,0,0,0,0,0
8,France,Les Natures,87,27.0,Roger Voss,Pinot Gris,Jean-Baptiste Adam,2012.0,Alsace,0,...,0,0,0,0,0,0,0,0,0,0
9,US,Mountain Cuvée,87,19.0,Virginie Boone,Cabernet Sauvignon,Kirkland Signature,2011.0,Napa Valley,1,...,0,0,0,0,0,0,0,0,0,0


In [110]:
col='variety'
cut = 0.9

In [111]:
n = data_copy[col].nunique()
cut = 0.9
m = data_copy.shape[0] * cut
counts = data_copy[col].value_counts()
su = 0
for i in range(n):
    su += counts[i]
    if su>m:
        ot = counts.index[i+1:]
        break
fil=[x in ot for x in data_copy[col]]
data_copy[col][fil]='other'        

dummy = pd.get_dummies(data_copy[col],dummy_na=True,prefix=col)

data_copy=data_copy.join(dummy)

data_copy=data_copy.drop(col,axis=1)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [112]:
data_copy

,country,designation,points,price,taster_name,winery,year,ex_title3,wine_nlp,flavor_nlp,...,variety_Tempranillo,variety_Tempranillo Blend,variety_Torrontés,variety_Verdejo,variety_Vermentino,variety_Viognier,variety_White Blend,variety_Zinfandel,variety_other,variety_nan
0,Portugal,Avidagos,87,15.0,Roger Voss,Quinta dos Avidagos,2011.0,Douro,1,0,...,0,0,0,0,0,0,0,0,0,0
1,US,NaN,87,14.0,Paul Gregutt,Rainstorm,2013.0,Willamette Valley,1,2,...,0,0,0,0,0,0,0,0,0,0
2,US,Reserve Late Harvest,87,13.0,Alexander Peartree,St. Julian,2013.0,Lake Michigan Shore,0,0,...,0,0,0,0,0,0,0,0,0,0
3,US,Vintner's Reserve Wild Child Block,87,65.0,Paul Gregutt,Sweet Cheeks,2012.0,Willamette Valley,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Spain,Ars In Vitro,87,15.0,Michael Schachner,Tandem,2011.0,Navarra,0,1,...,0,0,0,0,0,0,0,0,1,0
5,Italy,Belsito,87,16.0,Kerin O’Keefe,Terre di Giurfo,2013.0,Vittoria,0,0,...,0,0,0,0,0,0,0,0,1,0
6,France,NaN,87,24.0,Roger Voss,Trimbach,2012.0,Alsace,1,0,...,0,0,0,0,0,0,0,0,0,0
7,Germany,Shine,87,12.0,Anna Lee C. Iijima,Heinz Eifel,2013.0,Rheinhessen,1,1,...,0,0,0,0,0,0,0,0,0,0
8,France,Les Natures,87,27.0,Roger Voss,Jean-Baptiste Adam,2012.0,Alsace,0,1,...,0,0,0,0,0,0,0,0,0,0
9,US,Mountain Cuvée,87,19.0,Virginie Boone,Kirkland Signature,2011.0,Napa Valley,1,1,...,0,0,0,0,0,0,0,0,0,0


In [75]:
def to_dummy(data_copy,col,cut):
    n = data_copy[col].nunique()
    m = data_copy.shape[0] * cut
    counts = data_copy[col].value_counts()
    su = 0
    ot=[]
    for i in range(n):
        su += counts[i]
        if su>m:
            ot = counts.index[i+1:]
            break
    fil=[x in ot for x in data_copy[col]]
    data_copy[col][fil]='other'        

    dummy = pd.get_dummies(data_copy[col],dummy_na=True,prefix=col)
    data_copy=data_copy.join(dummy)
    data_copy=data_copy.drop(col,axis=1)  
    gc.collect()
    return data_copy

In [84]:
data=data_copy

In [135]:
data

,points,price,year,wine_nlp,flavor_nlp,fruit_nlp,finish_nlp,aroma_nlp,palat_nlp,acid_nlp,...,designation_del Comune di Barolo,designation_del Comune di Serralunga d'Alba,designation_g3,designation_other,designation_sbs,designation_Édition Limitée,designation_Ürziger Würzgarten Auslese,designation_Ürziger Würzgarten Kabinett,designation_Ürziger Würzgarten Spätlese,designation_nan
0,87,15.0,2011.0,1,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,87,14.0,2013.0,1,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,87,13.0,2013.0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,87,65.0,2012.0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,87,15.0,2011.0,0,1,1,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
5,87,16.0,2013.0,0,0,0,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
6,87,24.0,2012.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
7,87,12.0,2013.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,87,27.0,2012.0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,87,19.0,2011.0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
data=to_dummy(data,'country',0.9)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [87]:
data=to_dummy(data,'taster_name',1)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [88]:
data=to_dummy(data,'ex_title3',0.9)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [100]:
data=to_dummy(data,'variety',0.9)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [125]:
data['designation'].value_counts()[:4000].sum()

40805

In [110]:
data=to_dummy(data,'winery',0.5)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [126]:
data=to_dummy(data,'designation',0.3)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120975 entries, 0 to 120974
Columns: 5233 entries, points to designation_nan
dtypes: float64(2), int64(197), uint8(5034)
memory usage: 764.4 MB


In [51]:
data.to_csv('clean_data.csv')

In [129]:

with open('clean_data_pickle_fin','wb') as f:
    pickle.dump(data,f)

### PCA

In [2]:
with open('clean_data_pickle_fin','rb') as f:
    data = pickle.load(f)

In [3]:
data['year'].fillna(2011,inplace=True)

In [4]:
data['points'].fillna(88,inplace=True)

In [5]:
X=data.drop('price',axis=1)

In [38]:
Y=data['price']

In [39]:
with open('pca_y_pd','wb') as f:
    pickle.dump(Y,f)

In [6]:
print(X.shape)

(120975, 5232)


In [26]:
pca=0

In [27]:
pca = PCA(n_components=500)

In [28]:
gc.collect()

7

In [29]:
pca_X = pca.fit_transform(X)

In [32]:
pca_X = pd.DataFrame(pca_X)

In [33]:
pca_X.shape

(120975, 500)

In [35]:
pca_X

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [37]:
pca.explained_variance_ratio_

array([3.18858210e-01, 2.24353947e-01, 2.13818583e-02, 1.43840670e-02,
       1.27150335e-02, 1.03476652e-02, 9.98236883e-03, 8.72356120e-03,
       7.76156378e-03, 7.08053944e-03, 6.39903966e-03, 5.99124610e-03,
       5.64196711e-03, 5.44435390e-03, 4.99423066e-03, 4.90589232e-03,
       4.86495072e-03, 4.55647940e-03, 4.32511648e-03, 4.19508549e-03,
       4.09342116e-03, 4.04262363e-03, 3.89113671e-03, 3.72215402e-03,
       3.64346084e-03, 3.53896340e-03, 3.51306730e-03, 3.41795031e-03,
       3.28984879e-03, 3.12355692e-03, 3.05571930e-03, 2.97032579e-03,
       2.82518423e-03, 2.80493607e-03, 2.73752152e-03, 2.64714050e-03,
       2.62481736e-03, 2.57754565e-03, 2.51017155e-03, 2.48276734e-03,
       2.46397541e-03, 2.44688826e-03, 2.39453574e-03, 2.38175681e-03,
       2.35193693e-03, 2.32102561e-03, 2.26707690e-03, 2.24537241e-03,
       2.22779719e-03, 2.14570739e-03, 2.12494181e-03, 2.07525466e-03,
       2.06151047e-03, 2.03940301e-03, 2.01980792e-03, 1.99891524e-03,
      

In [40]:
with open('pca_500_x_pd','wb') as f:
    pickle.dump(pca_X,f)

In [41]:
with open('pca_500_model','wb') as f:
    pickle.dump(pca,f)

In [42]:
pca

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)